# Gather Energy Usage Data

In [7]:
import pandas as pd

In [8]:
# load the dataset
data = pd.read_json("data/sample.json")

In [9]:
# show sample of the data
data.head()

,prompt_text
0,how can identity protection services help prot...
1,Beside OFAC's selective sanction that target t...
2,You are the text completion model and you must...
3,The sum of the perimeters of three equal squar...
4,What is the type of the variables in the follo...


## Energy Measurement Verification
1. Run sample prompts through **CodeCarbon + LLM API**.  
2. Confirm energy data logs correctly and no major errors occur.  

---

## LLM Models Tested
- **GPT-4o / GPT-4o-mini** (OpenAI)  
- **Claude 3 Sonnet** (Anthropic)  
- **Mistral Large / Mixtral 8×22B** (Together.ai / Replicate)  
- **Gemini 1.5 Pro** (Google)


In [10]:
# Get energy data from LLM API calls
# TODO: Import codecarbon
# TODO: Import LLM libraries
# TODO: Set API keys

In [11]:
# Set up LLM clients
# TODO: OpenAI client
# TODO: Anthropic client
# TODO: Google client
# TODO: Together.ai client


In [12]:
# Track energy during API calls
# TODO: Energy tracking function
# TODO: Log energy data


In [13]:
# Make API calls to each model
# TODO: OpenAI function
# TODO: Anthropic function
# TODO: Google function
# TODO: Together.ai function


In [14]:
# Test energy with sample prompts
# TODO: Run tests
# TODO: Save energy data
